In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import xarray as xr
import itertools
import os
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt

In [ ]:
ds = r'c:\git\barotse\wflow\all_outputs_001\outmaps.nc'
ds = xr.open_dataset(ds)
ds

In [ ]:
# plot some spatial sums
import numpy as np
f = plt.figure(figsize=(16,16))
ssf = ds['ssf_riv'].sum(dim='time')
qo = ds['qo_riv'].sum(dim='time')
# Inwater = dss[-1]['Inwater'].sum(dim='time')
qo = qo.where(qo+ssf>0)
ssf = ssf.where(qo+ssf>0)
ssf *= ssf>0
qo *= qo>0
# watbal *=watbal<1750
# dss[-1]['wat'][25].plot()
ax1 = plt.subplot(221)
qo.plot(ax=ax1, cmap='viridis_r')

ax2 = plt.subplot(222)
ssf.plot(ax=ax2, cmap='viridis_r')

ax3 = plt.subplot(223)
(qo+ssf).plot(ax=ax3, cmap='viridis_r')

ax4 = plt.subplot(224)
ds['ex_ss'][53].plot(ax=ax4, cmap='Blues_r')
# wb = watbal.mean(dim=('lat', 'lon'))
# wb.plot()

In [ ]:
f = plt.figure(figsize=(16, 16))
ax1 = plt.subplot(221)
ax2 = plt.subplot(222)
ax3 = plt.subplot(223)
ax4 = plt.subplot(224)
ax1.grid()
ax2.grid()
ax3.grid()
ax4.grid()
surf_area = 5e5 * 1e6 # (about 500,000 km2)

conv_fact = 86400 * surf_area**-1 * 1e3

zi = -ds['zi'].mean(dim=('lat', 'lon'))
transfer = ds['trans'].mean(dim=('lat', 'lon'))
ssf = ds['ssf_riv'].sum(dim=('lat', 'lon'))
qo = ds['qo_riv'].sum(dim=('lat', 'lon'))
inw = ds['inw'].sum(dim=('lat', 'lon'))
surf_runoff = inw-ssf-qo
tot = ssf + qo

# make plots
(zi).plot(ax=ax1, label='Groundwater level')
ax1.set_ylabel('depth under surface [mm]')

#transfer.cumsum().plot(ax=ax2, label='Accumulated transfer from unsaturated to saturated zone')
#ax2.set_ylabel('Transfer [mm]')
#ax2.legend()

qo.plot(ax=ax3, label='lateral excess outflow from soil')
ssf.plot(ax=ax3, label='sub-surface lateral flow')
surf_runoff.plot(ax=ax3, label='runoff from surface')
ax3.set_ylabel('Lateral flux [m3/s]')
ax3.legend()

#inw.plot(ax=ax4, label='Total lateral flow per timestep')
#ax4.set_ylabel('Lateral flux [m3/s]')
#ax4.legend()

In [ ]:
# in mm
f = plt.figure(figsize=(16, 10))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
# ax3 = plt.subplot(223)
# ax4 = plt.subplot(224)
ax1.grid()
ax2.grid()
# ax3.grid()
# ax4.grid()
surf_area = 3580 * 3688 # 5e5 * 1e6 # (about 500,000 km2)
conv_fact = 86400 * surf_area**-1 * 1e3 # from m3/s to mm day-1
# ds['swf'][100].plot()
zi = -ds['zi'].mean(dim=('lat', 'lon'))
ssf = ds['ssf_riv'].mean(dim=('lat', 'lon'))*conv_fact  # from m3 s-1 to mm per day
qo = ds['qo_riv'].mean(dim=('lat', 'lon'))*conv_fact  # from m3 s-1 to mm per day
ex_us = ds['ex_us'].mean(dim=('lat', 'lon'))
ex_ss = ds['ex_ss'].mean(dim=('lat', 'lon'))
inw = ds['inw'].mean(dim=('lat', 'lon'))*conv_fact
P = ds['P'].mean(dim=('lat', 'lon'))
AET = ds['AET'].mean(dim=('lat', 'lon'))
# tot = ssf + qo
(zi).plot(ax=ax1)
plt.ylabel('depth under surface [mm]')
ssf.cumsum().plot(ax=ax2, label='subsurface flow')
qo.cumsum().plot(ax=ax2, label='land runoff')
ex_us.cumsum().plot(ax=ax2, label='exfilt_us')
ex_ss.cumsum().plot(ax=ax2, label='exfilt_ss')
inw.cumsum().plot(ax=ax2, label='total runoff')
P.cumsum().plot(ax=ax2, color='k', linestyle='-', label='Precip')
AET.cumsum().plot(ax=ax2, color='k', linestyle='--', label='Evaporation')
plt.ylabel('cumulative flux [mm]')
plt.legend()


In [ ]:
#lon = 23.2  # Lukulu
#lat = -14.337 # Lukulu
#lon = 24.15 # Watopa
#lat = -13.584 # Watopa
#lon = 23.248  # Senanga
#lat = -16.116 # Senanga
#lon = 22.67 # Chavuma
#lat = -13.083 # Chavuma
lon = 22.686 # Kalabo
lat = -14.988 # Kalabo

#ac = ds['accu'].sel(lat=lat, lon=lon, method='nearest')
ts = ds['run'].sel(lat=lat, lon=lon, method='nearest')
#ac.plot(label='instantaneous accumulated flow')
ts.plot(label='Routed discharge')
plt.ylabel('Discharge [m3/s]')
plt.legend()
plt.grid()


In [ ]:
ts_month = ts.resample(time='MS').mean()
#ac_month = ac.resample(time='MS').mean()

In [ ]:
#ac_month.plot(label='Instantaneous accumulated flow')
ts_month.plot(label='Routed discharge')
plt.ylabel('Discharge [m3/s]')
plt.legend()
plt.grid()


In [ ]:
fn = r'c:\zambezi_4km\observed_flow_calib.csv'
df = pd.read_csv(fn, parse_dates=True, index_col=[0])
#df
#df.plot()
monthly=df['2001-10-01':'2003-09-30']
monthly.plot()
#plt.xlabel('Year')
plt.legend(loc='center left', bbox_to_anchor=(0.99, 0.5))
plt.ylabel('Discharge m3/s')
plt.title('Upper Zambezi Observed Flows')
#plt.Figure(figsize=(15, 15))

In [ ]:
df.Kalabo

In [ ]:
plt.plot(df.Kalabo)
#ts.plot(label='Routed discharge')
ts_month.plot(label='Simulated discharge')
plt.title('Kalabo Gauge Station')
plt.ylabel('Discharge [m3/s]')
plt.legend('OS')
plt.grid()

#sim.plot()
#f = plt.figure(figsize=(15,3))
#plt.show()

-ROOT MEAN SQUARE ERROR (EMSE) ANALYSIS
-We want to loop through all results. So we make a template filename for the output.nc maps. Then within the loop we fill in the template parts.

In [ ]:
#rms = sqrt(mean_squared_error(df.Lukulu,ts))

rmses = np.sqrt(np.sum((obs-sim)**2))

In [2]:
root_folder = r'c:\git\barotse\wflow'
template = 'all_outputs_{:03d}\outmaps.nc'

KsatVerList = range(1, 2)
RootingDepthList = np.arange(1,4 )
SoilThicknessList = range(1, 4)
rmses = []
mult_Ksat = []
mult_root = []

for n, (KsatVer, RootingDepth, SoilThickness) in enumerate(itertools.product(KsatVerList,
                                                                                 RootingDepthList,
                                                                                 SoilThicknessList,
                                                                                )):
    path = os.path.join(root_folder,template.format(n + 1))
    print(path)
    ds = xr.open_dataset(path)
    mult_Ksat.append(KsatVer)
    #mult_root.append(RootingDepth)
    #mult_thickness.append(SoilThickness)
    
    # extract flows
    # compute rmse
    rmses.append(rmses)
    # put multipliers and rmse in a list
    # add list to a larger list

c:\git\barotse\wflow\all_outputs_001\outmaps.nc
c:\git\barotse\wflow\all_outputs_002\outmaps.nc
c:\git\barotse\wflow\all_outputs_003\outmaps.nc
c:\git\barotse\wflow\all_outputs_004\outmaps.nc
c:\git\barotse\wflow\all_outputs_005\outmaps.nc
c:\git\barotse\wflow\all_outputs_006\outmaps.nc
c:\git\barotse\wflow\all_outputs_007\outmaps.nc
c:\git\barotse\wflow\all_outputs_008\outmaps.nc
c:\git\barotse\wflow\all_outputs_009\outmaps.nc


In [ ]:
# ploit results
plt.plot(mult_Ksat, rmses, '.')
